<a href="https://colab.research.google.com/github/nunonegrier/confere-facil/blob/main/confere_facil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

In [42]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings
import re

warnings.filterwarnings("ignore")

In [9]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [10]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [50]:
############################################
# --- Agente 1: Pesquisador de fraudes --- #
############################################
def agente_buscador(descricao_ocorrencia):
  buscador = Agent(
      name="agente_buscador",
      model="gemini-2.0-flash",
      description="Agente que busca informações sobre a ocorrência",
      tools=[google_search],
      instruction="""
      Você é um pesquisador de segurança digital.
      Possui vastos conhecimentos nos mais diversos tipos de fraudes digitais sendo feitas atualmente.
      A sua tarefa é a de analisar a descrição da ocorrência abaixo, e, para complementar a análise, deve também utilizar
      a ferramenta de busca do google (google_search) para ter acesso às informações mais recentes sobre o tema.
      A análise deve calcular a probabilidade de ser uma tentativa de fraude em curso assim como identificar o possível tipo de fraude.
      De preferência, deverá fazer uma identificação única e acertiva do tipo de fraude em andamento, mas, não sendo possível, deverá listar
      uma segunda possível identificação do tipo de fraude.
      Faça apenas a análise sem apresentar recomendações.
      Dê uma resposta suscinta, mas que não deixe nenhuma informação importante de fora.
      """
  )

  entrada_do_agente_de_pesquisa_inicial = f"Ocorrência: {descricao_ocorrencia}"

  # Executa o agente
  pesquisa_inicial = call_agent(buscador, entrada_do_agente_de_pesquisa_inicial)
  return pesquisa_inicial

In [51]:
################################################
#     --- Agente 2: Analista de fraudes ---    #
################################################
def agente_analista(descricao_ocorrencia, pesquisa_ocorrencia, respostas_usuario=None):
    analista = Agent(
        name="agente_analista",
        model="gemini-2.0-flash",
        instruction="""
        Você é um especialista em análise de fraudes.
        Com base na pesquisa inicial sobre a descrição da fraude:
        Crie entre 1 a 3 perguntas a serem feitas ao usuário e baseadas na pesquisa inicial, que auxiliem na análise da pesquisa inicial
        e retorne APENAS as perguntas numeradas.
        Se respostas do usuário forem fornecidas, utilize a pesquisa inicial e as respostas fornecidas para refinar a análise inicial,
        NÃO faça novas perguntas, use a ferramenta de busca do google (google_search) para buscar informações mais recentes se forem necessárias
        e apresente a análise final suscinta, mas que não deixe nenhuma informação importante de fora.
        """,
        description="Agente que analisa fraudes",
        tools=[google_search]
    )

    entrada_do_agente_analista = f"Descrição:{descricao_ocorrencia}\nPesquisa: {pesquisa_ocorrencia}"
    if respostas_usuario:
        entrada_do_agente_analista += f"\nRespostas do usuário: {respostas_usuario}"

    # Executa o agente
    analise_da_fraude = call_agent(analista, entrada_do_agente_analista)
    return analise_da_fraude

In [52]:
####################################################
# --- Agente 3: Especialista em Cibersegurança --- #
####################################################
def agente_especialista(descricao_ocorrencia, analise_da_fraude):
    especialista = Agent(
        name="agente_especialista",
        model="gemini-2.0-flash",
        instruction="""
            Você é um especialista em cibersegurança.
            Com base na análise da fraude você deve:
            Fornecer instruções claras para que o usuário evite a fraude.
            Fornecer os contados dos canais de atendimento da empresa ou serviço, públicos ou privados, ou qualquer
            outro tipo de negócio que tenham sido mencionados na descrição da ocorrência para que o usuário possa entrar
            em contato para esclarecer a situação.
            """,
        description="Agente especialista de posts engajadores para Instagram",
        tools=[google_search]
    )
    entrada_do_agente_especialista = f"Descrição: {descricao_ocorrencia}\nAnálise: {analise_da_fraude}"

    # Executa o agente
    recomendacoes_finais = call_agent(especialista, entrada_do_agente_especialista)
    return recomendacoes_finais

In [54]:
print("🚀 Iniciando o Sistema de Análise e Prevenção de Fraudes 🚀")

# --- Obter a Descrição do Usuário ---
print("❓ Por favor, faça uma descrição da possível fraude:")
descricao_ocorrencia = input("")


if not descricao_ocorrencia:
  print("você esqueceu de digitar a descrição!")
else:
  print(f"Obrigado! Vamos analisar a ocorrência como foi descrita: {descricao_ocorrencia}")

  # 1o Agente
  pesquisa_ocorrencia = agente_buscador(descricao_ocorrencia)
  print("\n--- Resultado do Agente 1 (Buscador) ---\n")
  display(to_markdown(pesquisa_ocorrencia))
  print("-------------------------------------------------------------------")

  # 2o Agente
  # Primeira chamada do agente analista para obter as perguntas
  perguntas_texto = agente_analista(descricao_ocorrencia, pesquisa_ocorrencia)
  print("\n--- Perguntas do Agente 2 (Analista) ---\n")
  display(to_markdown(perguntas_texto))
  print("-------------------------------------------------------------------")

  # Extrair as perguntas numeradas do texto
  perguntas_lista = re.findall(r'\d+\.\s+(.*)', perguntas_texto)

  respostas_usuario = {} # Dicionário para armazenar as respostas {numero_pergunta: resposta}

  if perguntas_lista:
      print("\n❓ Por favor, responda às perguntas acima para ajudar na análise:")
      for i, pergunta in enumerate(perguntas_lista):
          print(f"Resposta para a pergunta {i + 1}:")
          print(f"'{pergunta.strip()}'")
          resposta = input("")
          respostas_usuario[f"Pergunta {i + 1}"] = resposta # Armazena a resposta associada ao número da pergunta
  else:
      print("Nenhuma pergunta foi gerada pelo agente analista.")

  # Preparar as respostas para enviar ao agente na segunda chamada
  respostas_formatadas = "\n".join([f"{chave}: {valor}" for chave, valor in respostas_usuario.items()])

  # Segunda chamada do agente analista com as respostas do usuário
  analise_da_fraude = agente_analista(descricao_ocorrencia, pesquisa_ocorrencia, respostas_usuario=respostas_usuario)
  print("\n--- Análise final do Agente 2 (Analista) ---\n")
  display(to_markdown(analise_da_fraude))
  print("-------------------------------------------------------------------")

  # 3o Agente
  recomendacoes_finais = agente_especialista(descricao_ocorrencia, analise_da_fraude)
  print("\n--- Resultado do Agente 3 (Especialista) ---\n")
  display(to_markdown(recomendacoes_finais))
  print("-------------------------------------------------------------------")

🚀 Iniciando o Sistema de Análise e Prevenção de Fraudes 🚀
❓ Por favor, faça uma descrição da possível fraude:
recebi um pedido de dinheiro emprestado de uma amigo no whatsapp, tenho receio de que não seja ele e que o seu whatsapp tenha sido roubado
Obrigado! Vamos analisar a ocorrência como foi descrita: recebi um pedido de dinheiro emprestado de uma amigo no whatsapp, tenho receio de que não seja ele e que o seu whatsapp tenha sido roubado

--- Resultado do Agente 1 (Buscador) ---



> Compreendo sua preocupação em relação ao pedido de dinheiro via WhatsApp. Para analisar a probabilidade de ser uma fraude, farei algumas buscas para complementar a análise.
> 
> 
> Com base na sua descrição e nas informações encontradas, a probabilidade de ser uma tentativa de fraude é alta. O tipo de fraude mais provável é o **"Golpe do WhatsApp"**, especificamente a variação em que criminosos se passam por conhecidos para pedir dinheiro.
> 
> Uma segunda possibilidade é que a conta do seu amigo tenha sido clonada, e os criminosos estão usando o número dele para pedir dinheiro a contatos.
> 
> 


-------------------------------------------------------------------

--- Perguntas do Agente 2 (Analista) ---



> Para refinar a análise e confirmar se é uma tentativa de fraude, preciso de mais informações. Por favor, responda às seguintes perguntas:
> 
> 1.  Você poderia me informar qual o valor solicitado e qual a justificativa apresentada para o empréstimo?
> 2.  Você já tentou confirmar a situação diretamente com seu amigo por outro meio de comunicação (ligação telefônica, mensagem em outra rede social, etc.)?
> 3.  O número de telefone que está solicitando o dinheiro é o mesmo que você tem salvo para o seu amigo?
> 


-------------------------------------------------------------------

❓ Por favor, responda às perguntas acima para ajudar na análise:
Resposta para a pergunta 1:
'Você poderia me informar qual o valor solicitado e qual a justificativa apresentada para o empréstimo?'
789,56, ele disse que esqueceu os cartões em casa e precisa pagar uma conta com urgência
Resposta para a pergunta 2:
'Você já tentou confirmar a situação diretamente com seu amigo por outro meio de comunicação (ligação telefônica, mensagem em outra rede social, etc.)?'
tentei ligar para ele mas não atendeu, só respondeu por mensagem de texto
Resposta para a pergunta 3:
'O número de telefone que está solicitando o dinheiro é o mesmo que você tem salvo para o seu amigo?'
sim

--- Análise final do Agente 2 (Analista) ---



> Com base na sua descrição inicial, nas respostas fornecidas e na minha pesquisa prévia, a probabilidade de ser uma tentativa de fraude é ainda maior. Os elementos que reforçam essa análise são:
> 
> 1.  **Pedido de valor específico e com urgência:** Golpistas frequentemente usam essa tática para pressionar a vítima a agir sem pensar.
> 2.  **Dificuldade em contatar o amigo por outros meios:** A indisponibilidade para atender ligações e a comunicação exclusiva por mensagens de texto são características comuns de golpes, pois o fraudador tenta evitar ser descoberto pela voz.
> 3.  **Alegada emergência (esquecimento dos cartões):** Essa é uma desculpa comum usada para justificar o pedido de dinheiro imediato.
> 
> **Análise Final:**
> 
> Diante dos fatos, tudo indica que você está sendo alvo do "Golpe do WhatsApp". Recomenda-se fortemente que **não faça nenhuma transferência** e tente confirmar a situação diretamente com seu amigo por outros meios de comunicação (ligação telefônica, contato pessoal, etc.). Adicionalmente, alerte seu amigo sobre a possível clonagem/invasão da conta dele para que ele possa tomar as medidas de segurança necessárias e avisar seus outros contatos.


-------------------------------------------------------------------

--- Resultado do Agente 3 (Especialista) ---



> Com certeza, aqui estão algumas instruções claras para evitar cair nesse tipo de golpe e os contatos que podem ser úteis:
> 
> **Instruções para Evitar a Fraude:**
> 
> 1.  **Confirme a Identidade:**
>     *   **Sempre ligue para o seu amigo(a)** por um número de telefone que você já tenha salvo, **antes de fazer qualquer transferência**.
>     *   Se não conseguir contato telefônico, tente falar com ele(a) pessoalmente ou por meio de outro contato que você tenha (outro amigo em comum, familiar, etc.).
> 2.  **Desconfie de Urgência e Segredos:**
>     *   Golpistas frequentemente criam um senso de urgência para que você não tenha tempo de pensar ou verificar a situação.
>     *   Desconfie se pedirem para você não contar nada a ninguém.
> 3.  **Verifique a Gramática e a Redação:**
>     *   Preste atenção na forma como a pessoa escreve. Golpistas podem cometer erros de gramática ou usar uma linguagem diferente da que seu amigo(a) usaria.
> 4.  **Não Forneça Informações Pessoais:**
>     *   Nunca compartilhe senhas, códigos de verificação ou informações bancárias por WhatsApp.
> 5.  **Ative a Verificação em Duas Etapas no WhatsApp:**
>     *   Isso adiciona uma camada extra de segurança à sua conta. Para ativar, vá em Configurações > Conta > Verificação em duas etapas.
> 6.  **Desconfie de Pedidos de Transferência para Contas de Terceiros:**
>     *   Se o pedido de dinheiro envolver uma conta bancária que não seja do seu amigo(a), desconfie ainda mais.
> 7.  **Mantenha a Calma:**
>     *   Golpistas querem que você entre em pânico. Respire fundo e siga os passos acima antes de tomar qualquer atitude.
> 8.  **Informe seus Contatos:**
>     *   Se você descobrir que um amigo(a) foi hackeado, avise outros contatos dele(a) para que não caiam no golpe.
> 9.  **Faça um Boletim de Ocorrência:**
>     *   Se você foi vítima de um golpe, registre um boletim de ocorrência na polícia.
> 
> **Canais de Atendimento e Contato:**
> 
> Como a descrição da ocorrência não menciona nenhuma empresa ou serviço específico, não é possível fornecer contatos de canais de atendimento. No entanto, se você tiver informações adicionais sobre a empresa ou serviço envolvido, posso tentar encontrar os contatos relevantes.
> 
> **O mais importante neste momento é:**
> 
> *   **Não faça a transferência solicitada.**
> *   **Tente entrar em contato com seu amigo(a) por outros meios** para confirmar se ele(a) realmente precisa do dinheiro.
> *   **Avise seu amigo(a)** sobre a possível invasão da conta dele(a) para que ele(a) possa tomar as medidas de segurança necessárias.
> 


-------------------------------------------------------------------
